In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')
mx = df.ts.max()
df = df.loc[df.ts >= mx-60*60*24*21]

df['hour'] = df.ts % (60*60*24)

df = df.loc[ (df.hour < 60*60*14)&(df.hour >= 60*60*2) ]

df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(71604095, 6)


,session,aid,ts,type,hour,day
63,0,1629651,1659964140,0,47340,392940
64,0,1768568,1660015113,0,11913,443913
65,0,1318324,1660015128,0,11928,443928
66,0,1840418,1660024261,0,21061,453061
67,0,1813509,1660024355,0,21155,453155


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
63,0,1629651,1659964140,0,47340,392940,<NA>,-138917,0,0
64,0,1768568,1660015113,0,11913,443913,138917,450244,0,0
65,0,1318324,1660015128,0,11928,443928,-450244,-522094,0,0
66,0,1840418,1660024261,0,21061,453061,522094,26909,0,0
67,0,1813509,1660024355,0,21155,453155,-26909,0,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
354645     <NA>
1127337    <NA>
1569324     1.0
1575767     3.0
1733477     1.0
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(194.5, 289.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

113

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
18277,4,3,1,1,-1.0,-1.0
18282,49,47,1,7,2.0,1.0
18287,11,12,1,2,-1.0,-1.0
18285,10,11,0,2,-1.0,1.0
18286,51,56,0,2,-1.0,1.0


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,17,15,32152.411765,387917.117647
1,0.000000,16,13,34758.625000,196758.625000
2,0.000000,13,12,40367.384615,279628.923077
3,0.088407,1199,669,36423.236864,302252.761468
4,0.047619,63,48,36173.492063,296744.920635


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
1782029,0.394771,9268.688477,180363.000000
536462,0.541708,9963.676758,155027.703125
637223,0.000000,2900.231201,2900.231201
1402232,0.858360,8592.741211,193286.921875
902006,0.741882,11491.552734,191471.484375


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,17,15,32152.412109,387917.12500,1.133333,0.000000,10038.110352,152401.703125,13,15,0,0,-1.000000,-1.000000
1,0.000000,16,13,34758.625000,196758.62500,1.230769,0.000000,11851.911133,133107.281250,15,13,0,0,-1.000000,-1.000000
2,0.000000,13,12,40367.382812,279628.93750,1.083333,0.000000,6893.389160,134957.984375,9,10,0,0,-1.000000,-1.000000
3,0.088407,1199,669,36423.238281,302252.75000,1.792227,0.322538,9016.490234,144964.500000,182,171,14,78,2.333333,2.090909
4,0.047619,63,48,36173.492188,296744.90625,1.312500,0.214669,9102.734375,149799.468750,45,43,0,3,-1.000000,1.000000


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}9' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio9', 'count_item9', 'count_user9', 'hour_mean9', 'day_mean9', 'repeat9', 'buy_ratio_std9', 'hour_std9', 'day_std9', 'prev9', 'next9', 'orders9', 'carts9', 'order_repeat9', 'cart_repeat9']


In [19]:
item_features.to_parquet('../../data/item_user_features/item22_LB.pqt')